In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from nltk.stem import PorterStemmer
import pandas as pd
import re
from sklearn.model_selection import train_test_split
import jieba
from gensim.models import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile
import sklearn
import scipy
from gensim.test.utils import datapath
from gensim.corpora import Dictionary
from gensim.corpora import MalletCorpus
from gensim.test.utils import get_tmpfile, common_texts
from gensim.models.wrappers import LdaMallet
from gensim.test.utils import common_corpus, common_dictionary
import time
from collections import defaultdict
from random import random, randint
from glob import glob
from math import log
import argparse
import random

from nltk.corpus import stopwords
from nltk.probability import FreqDist

from nltk.tokenize import TreebankWordTokenizer
kTOKENIZER = TreebankWordTokenizer()
kDOC_NORMALIZER = True

In [65]:
search_path = "%s/*.txt" % './w2v/lit/en'

files = glob(search_path)

documents = []

for i in files:
    contents = open(i).read()
    documents.append(contents)


texts = [
    [word for word in jieba.cut(document)] for document in documents
]


print(len(texts))

model_en = Word2Vec(texts, size=100, window=5, min_count=1, workers=4)
model_en.train(texts,total_examples=model.corpus_count, epochs=100)


1


(1000000, 34807500)

In [66]:
search_path = "%s/*.txt" % './w2v/lit/cn'

files = glob(search_path)

documents = []

for i in files:
    contents = open(i).read()
    documents.append(contents)


texts = [
    [word for word in jieba.cut(document)] for document in documents
]


print(len(texts))

model_cn = Word2Vec(texts, size=100, window=5, min_count=1, workers=4)
model_cn.train(texts,total_examples=model.corpus_count, epochs=100)


1


(1000000, 15105900)

In [69]:
def get_similar_vector(model_i,word):
    try:
        word = model_i.wv.most_similar(positive=[model_i.wv[word]], topn=2)[1][0]
        most_similar_wv = model_i.wv[word]
    except:
        most_similar_wv = model_i.wv[random.choice(model_i.wv.index2entity)]
    return most_similar_wv

In [81]:

# try:
#     word = model.wv.most_similar(positive=[model.wv['pride']], topn=2)[1][0]
#     most_similar_wv = model.wv[word]
# except:
#     most_similar_wv = model.wv[random.choice(model.wv.index2entity)]
    
print(get_similar_vector(model_en,'pridedd'))

[-3.6263696e-03  2.2747482e-03 -4.1053354e-04  3.6275804e-03
  1.2554936e-03 -3.9693443e-03 -1.7822701e-04  1.3358082e-03
 -1.9464294e-03  2.7665666e-03 -1.1480190e-03  1.3480812e-03
  3.7503797e-03 -1.8680529e-03 -1.9776255e-03 -2.7777026e-03
  1.6252104e-03  4.7658924e-03  3.6692990e-03  4.4510602e-03
  2.0423895e-03 -3.7964901e-03 -1.5479241e-03  9.3749084e-04
  1.9532603e-03  1.8262930e-03 -4.6424796e-03  2.3688853e-03
  1.5256767e-03 -1.8302690e-03 -2.7226042e-03 -3.5747819e-04
  4.4828053e-03  2.0401783e-03 -5.5992516e-04 -1.8783150e-03
  4.3989369e-03 -3.2833919e-03 -2.4042081e-03  1.1698157e-03
 -2.6866526e-03  4.2988225e-03  3.3111698e-03 -7.4235606e-04
  3.5434638e-03 -4.7810171e-03 -1.4192410e-03 -2.1528089e-03
  4.4101290e-03  2.0722512e-03  4.2216320e-04  8.4836257e-04
  5.7819876e-04 -4.9428041e-03 -1.5675620e-04  1.6696197e-03
 -1.4490996e-03  1.9065524e-03 -2.6394129e-03 -3.3217783e-03
  2.6886594e-03  9.1919173e-05  5.0500274e-04  3.3762597e-03
 -3.4516463e-03 -4.26878

In [ ]:
tw_pd_en = pd.read_csv("./base_line_classifier/tra.csv",sep=',',encoding='latin-1',names=['class','ram','time','q','user_id','text'])

train_S = tw_pd_en['class']
train_ST = tw_pd_en['text']

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text


def tokenizer(text):
    words = text.split()
    words = [tuple(get_similar_vector(model_en,word)) for word in words]
    return words

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]



# tokenizer('A good day')

In [ ]:
stop = stopwords.words('english')

# split the dataset in train and test
X = train_ST
y = train_S
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__tokenizer': [tokenizer],
               'vect__preprocessor': [ preprocessor],
               'clf__penalty': ['l2'],
               'clf__C': [1.0]},
              ]

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

gs_lr_tfidf.fit(X_train, y_train)

print('Best parameter set: ' + str(gs_lr_tfidf.best_params_))
print('Best accuracy: %.3f' % gs_lr_tfidf.best_score_)

clf = gs_lr_tfidf.best_estimator_
print('Accuracy in test: %.3f' % clf.score(X_test, y_test))


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
